# Environment

This notebook sets up a PyBullet environment for the [OpenAI Gym](https://gym.openai.com/) interface. The environment is based on the [PyBullet](https://pybullet.org/wordpress/) physics simulator. The environment is a satellite orbiting a moon. The satellite is controlled by a thruster that can be fired in the x, y, or z direction. The goal is to keep the satellite in a stable orbit around the moon.

In [1]:
import pybullet as p
import pybullet_data
import time
import numpy as np

pybullet build time: Oct 14 2023 16:00:11


In [2]:
# Constants
sim_speed = 480
G = 6.67430e-11

r_moon = 2.52e2
m_moon = 1.00e14
x_moon = np.array([0, 0, 0])

r_satellite = 1.00
m_satellite = 1.00e3
x_satellite = np.array([2 * r_moon, 0, 0])

In [3]:
# Create the client
physicsClient = p.connect(p.GUI)
p.setAdditionalSearchPath(pybullet_data.getDataPath())

# Set simulation speed
p.setRealTimeSimulation(0)
p.setTimeStep(sim_speed * 1.0 / 240.0)

Version = 4.1 Metal - 87
Vendor = Apple
Renderer = Apple M1 Max
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started


In [4]:
p.loadURDF("plane.urdf")

# Load the moon
moonCollider = p.createCollisionShape(p.GEOM_SPHERE, radius=r_moon)
moonVisual   = p.createVisualShape(p.GEOM_SPHERE, radius=r_moon, rgbaColor=[0.5, 0.5, 0.5, 1])
moon = p.createMultiBody(baseMass=m_moon, baseCollisionShapeIndex=moonCollider, baseVisualShapeIndex=moonVisual, basePosition=x_moon)

# Load the satellite
satelliteCollider = p.createCollisionShape(p.GEOM_SPHERE, radius=r_satellite)
satelliteVisual   = p.createVisualShape(p.GEOM_SPHERE, radius=r_satellite, rgbaColor=[0, 0, 1, 1])
satellite = p.createMultiBody(baseMass=m_satellite, baseCollisionShapeIndex=satelliteCollider, baseVisualShapeIndex=satelliteVisual, basePosition=x_satellite)

# Set the camera position
p.resetDebugVisualizerCamera(cameraDistance=3 * r_moon, cameraYaw=0, cameraPitch=-90, cameraTargetPosition=[0, 0, 0])

# Set the gravity
p.setGravity(0, 0, 0)

# Set the initial velocity
v = np.sqrt(G * m_moon / r_moon)
v_satellite = np.array([0, v, 0])
p.resetBaseVelocity(satellite, v_satellite)

# Run the simulation
for i in range(100000):
    p.stepSimulation()
    time.sleep(1.0 / sim_speed)
    x_satellite, _ = p.getBasePositionAndOrientation(satellite)
    x_moon, _ = p.getBasePositionAndOrientation(moon)
    r = np.array(x_satellite) - np.array(x_moon)
    r_norm = np.linalg.norm(r)
    a = -G * m_moon / r_norm ** 3 * r
    p.applyExternalForce(satellite, -1, a * m_satellite, x_satellite, p.WORLD_FRAME)

p.disconnect()